1. 导入模块将wav无损音频分离到audio文件夹中

In [ ]:
import os
from tqdm import tqdm, trange
import time
from moviepy.editor import *
video_path="./video"
audio_path="./audio"

In [24]:
video_list=os.listdir(video_path)
#print(video_list)
for file in tqdm((video_list),desc='processing'):
    if('.mp4' in file):
        video = VideoFileClip(video_path+"/"+file)
        audio = video.audio
        #print(file.replace(".mp4", ".wav"))
        (audio.write_audiofile(audio_path+'/'+file.replace(".mp4", ".wav")))

processing: 100%|██████████| 20/20 [00:07<00:00,  2.75it/s]         


MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR1_ch06.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR0_ch06.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR0_ch07.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR1_ch07.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR0_ch05.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR1_ch05.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR0_ch10.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR1_ch04.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR1_ch10.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR0_ch04.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR1_ch01.wav
MoviePy - Done.
MoviePy - Writing audio in ./audio/gBR_sBM_c01_d04_mBR0_ch01.wav
MoviePy - Done.
MoviePy - Writing audio in .

2.使用librosa进行音频处理


In [25]:
import librosa
